In [35]:
import pandas as pd
import numpy as np
from scipy import stats

In [36]:
# Load the data to get cluster 4 IDs
demo_df = pd.read_csv(r"C:\Users\sonis\Downloads\comp_socio_df.csv")
cluster4_ids = demo_df[demo_df['Cluster_Comp06_k07'] == 4]['ANON_ID'].tolist()

In [37]:
# Load the electricity data 
print("Electricity data")
elec_df = pd.read_csv(r"C:\Users\sonis\carbon_full_day.csv.gz", compression='gzip')
print(f"Total electricity records: {len(elec_df)}")

Electricity data
Total electricity records: 24246420


In [38]:
# Filter for cluster 4 customers only
cluster4_elec = elec_df[elec_df['ANON_ID'].isin(cluster4_ids)]
print(f"Cluster 4 electricity records: {len(cluster4_elec):,}")
print(f"Unique cluster 4 customers found: {cluster4_elec['ANON_ID'].nunique()}")

# Fix the copy warning and prepare data
cluster4_elec = cluster4_elec.copy()
cluster4_elec['DateTime'] = pd.to_datetime(cluster4_elec['DateTime'])

# Filter for April-December analysis periods only
cluster4_elec['month'] = cluster4_elec['DateTime'].dt.month
cluster4_elec['year'] = cluster4_elec['DateTime'].dt.year
cluster4_elec['hour'] = cluster4_elec['DateTime'].dt.hour
cluster4_elec['day_of_week'] = cluster4_elec['DateTime'].dt.dayofweek  # 0=Monday, 6=Sunday

# Filter for April-December in both years
analysis_data = cluster4_elec[cluster4_elec['month'].isin([4,5,6,7,8,9,10,11,12])].copy()

print("Cluster  4 Analysis")
print(f"Analysis period data: {len(analysis_data):,} records")

Cluster 4 electricity records: 2,672,251
Unique cluster 4 customers found: 98
Cluster  4 Analysis
Analysis period data: 2,257,094 records


In [39]:
# PEAK HOURS kWh ANALYSIS (4-7pm, Mon-Sat)

print("\nPEAK HOURS kWh ANALYSIS")

# Filter for peak hours: 4-7pm (16-19), Monday-Saturday (0-5)
peak_data = analysis_data[
    (analysis_data['hour'].isin([16, 17, 18, 19])) &  # 4pm-7pm
    (analysis_data['day_of_week'].isin([0, 1, 2, 3, 4, 5]))  # Mon-Sat
].copy()

print(f"Peak hours data: {len(peak_data):,} records")

# Calculate daily peak totals per customer
daily_peak_totals = peak_data.groupby([
    'ANON_ID', 'treatment', 'period', 
    peak_data['DateTime'].dt.date
]).agg({
    'ELEC_KWH': 'sum'  # Daily peak kWh total (only 4-7pm)
}).reset_index()

daily_peak_totals.columns = ['ANON_ID', 'treatment', 'period', 'date', 'daily_peak_kwh']
daily_peak_totals['date'] = pd.to_datetime(daily_peak_totals['date'])
daily_peak_totals['year_month'] = daily_peak_totals['date'].dt.to_period('M')

print(f"Daily peak totals calculated: {len(daily_peak_totals):,} customer-days")

# Calculate monthly averages for peak hours
monthly_peak_avg = daily_peak_totals.groupby([
    'ANON_ID', 'treatment', 'period', 'year_month'
]).agg({
    'daily_peak_kwh': 'mean'
}).reset_index()

print(f"Monthly peak averages calculated: {len(monthly_peak_avg):,} customer-months")


PEAK HOURS kWh ANALYSIS
Peak hours data: 345,938 records
Daily peak totals calculated: 43,459 customer-days
Monthly peak averages calculated: 1,711 customer-months


In [41]:
# FULL DAY CO2 ANALYSIS (All 48 intervals)

print("\nFULL DAY CO2 ANALYSIS")

# Calculate daily CO2 totals per customer (all day)
daily_co2_totals = analysis_data.groupby([
    'ANON_ID', 'treatment', 'period', 
    analysis_data['DateTime'].dt.date
]).agg({
    'carbon_emissions': 'sum'  # Daily CO2 total (all 48 intervals)
}).reset_index()

daily_co2_totals.columns = ['ANON_ID', 'treatment', 'period', 'date', 'daily_co2_grams']
daily_co2_totals['date'] = pd.to_datetime(daily_co2_totals['date'])
daily_co2_totals['year_month'] = daily_co2_totals['date'].dt.to_period('M')

print(f"Daily CO2 totals calculated: {len(daily_co2_totals):,} customer-days")

# Calculate monthly averages for CO2
monthly_co2_avg = daily_co2_totals.groupby([
    'ANON_ID', 'treatment', 'period', 'year_month'
]).agg({
    'daily_co2_grams': 'mean'
}).reset_index()

print(f"Monthly CO2 averages calculated: {len(monthly_co2_avg):,} customer-months")


FULL DAY CO2 ANALYSIS
Daily CO2 totals calculated: 47,813 customer-days
Monthly CO2 averages calculated: 1,711 customer-months


In [44]:
# PEAK kWh GROUP ANALYSIS

# Calculate monthly group averages for PEAK kWh
monthly_peak_group_avg = monthly_peak_avg.groupby(['treatment', 'period', 'year_month']).agg({
    'daily_peak_kwh': ['mean', 'std', 'count']
}).reset_index()

monthly_peak_group_avg.columns = ['treatment', 'period', 'year_month', 'mean_peak_kwh', 'std_peak_kwh', 'count_peak_kwh']

# Create comparison dataset for peak kWh
pre_peak_data = monthly_peak_group_avg[monthly_peak_group_avg['period'] == 'Pre'].copy()
post_peak_data = monthly_peak_group_avg[monthly_peak_group_avg['period'] == 'Post'].copy()

peak_comparison = pd.merge(
    pre_peak_data[['treatment', 'year_month', 'mean_peak_kwh']],
    post_peak_data[['treatment', 'year_month', 'mean_peak_kwh']],
    on=['treatment'],
    suffixes=('_pre', '_post')
)

# Calculate peak kWh changes
peak_comparison['peak_kwh_change'] = peak_comparison['mean_peak_kwh_post'] - peak_comparison['mean_peak_kwh_pre']
peak_comparison['peak_kwh_pct_change'] = (peak_comparison['peak_kwh_change'] / peak_comparison['mean_peak_kwh_pre']) * 100

# Calculate peak intervention effect
control_peak_changes = peak_comparison[peak_comparison['treatment'] == 'Control']
intervention_peak_changes = peak_comparison[peak_comparison['treatment'] == 'Intervention']

peak_effects = pd.DataFrame({
    'month_pre': control_peak_changes['year_month_pre'].values,
    'month_post': intervention_peak_changes['year_month_post'].values,
    'control_peak_kwh_change': control_peak_changes['peak_kwh_change'].values,
    'intervention_peak_kwh_change': intervention_peak_changes['peak_kwh_change'].values,
})

peak_effects['net_peak_kwh_reduction'] = (
    peak_effects['intervention_peak_kwh_change'] - peak_effects['control_peak_kwh_change']
)

control_baseline_peak_kwh = control_peak_changes['mean_peak_kwh_pre'].values
peak_effects['net_peak_kwh_pct_reduction'] = (
    peak_effects['net_peak_kwh_reduction'] / control_baseline_peak_kwh
) * 100

print("\nPEAK kWh RESULTS")
print(f"Average daily PEAK kWh reduction: {peak_effects['net_peak_kwh_reduction'].mean():.3f} kWh/day")
print(f"Average percentage PEAK kWh reduction: {peak_effects['net_peak_kwh_pct_reduction'].mean():.2f}%")


PEAK kWh RESULTS
Average daily PEAK kWh reduction: -0.211 kWh/day
Average percentage PEAK kWh reduction: -8.48%


In [45]:
# FULL DAY CO2 GROUP ANALYSIS

# Calculate monthly group averages for FULL DAY CO2
monthly_co2_group_avg = monthly_co2_avg.groupby(['treatment', 'period', 'year_month']).agg({
    'daily_co2_grams': ['mean', 'std', 'count']
}).reset_index()

monthly_co2_group_avg.columns = ['treatment', 'period', 'year_month', 'mean_co2', 'std_co2', 'count_co2']

# Create comparison dataset for CO2
pre_co2_data = monthly_co2_group_avg[monthly_co2_group_avg['period'] == 'Pre'].copy()
post_co2_data = monthly_co2_group_avg[monthly_co2_group_avg['period'] == 'Post'].copy()

co2_comparison = pd.merge(
    pre_co2_data[['treatment', 'year_month', 'mean_co2']],
    post_co2_data[['treatment', 'year_month', 'mean_co2']],
    on=['treatment'],
    suffixes=('_pre', '_post')
)

# Calculate CO2 changes
co2_comparison['co2_change'] = co2_comparison['mean_co2_post'] - co2_comparison['mean_co2_pre']
co2_comparison['co2_pct_change'] = (co2_comparison['co2_change'] / co2_comparison['mean_co2_pre']) * 100

# Calculate CO2 intervention effect
control_co2_changes = co2_comparison[co2_comparison['treatment'] == 'Control']
intervention_co2_changes = co2_comparison[co2_comparison['treatment'] == 'Intervention']

co2_effects = pd.DataFrame({
    'month_pre': control_co2_changes['year_month_pre'].values,
    'month_post': intervention_co2_changes['year_month_post'].values,
    'control_co2_change': control_co2_changes['co2_change'].values,
    'intervention_co2_change': intervention_co2_changes['co2_change'].values,
})

co2_effects['net_co2_reduction'] = (
    co2_effects['intervention_co2_change'] - co2_effects['control_co2_change']
)

control_baseline_co2 = control_co2_changes['mean_co2_pre'].values
co2_effects['net_co2_pct_reduction'] = (
    co2_effects['net_co2_reduction'] / control_baseline_co2
) * 100

print("\nFULL DAY CO2 RESULTS")
print(f"Average daily CO2 reduction: {co2_effects['net_co2_reduction'].mean():.1f} grams/day")
print(f"Average percentage CO2 reduction: {co2_effects['net_co2_pct_reduction'].mean():.2f}%")


FULL DAY CO2 RESULTS
Average daily CO2 reduction: -28.0 grams/day
Average percentage CO2 reduction: -1.73%


In [61]:
# Statistical significance testing
print("\nSTATISTICAL SIGNIFICANCE")
peak_kwh_tstat, peak_kwh_pval = stats.ttest_1samp(peak_effects['net_peak_kwh_reduction'], 0)
co2_tstat, co2_pval = stats.ttest_1samp(co2_effects['net_co2_reduction'], 0)

print(f"PEAK kWh reduction significance: t={peak_kwh_tstat:.3f}, p={peak_kwh_pval:.4f}")
print(f"FULL DAY CO2 reduction significance: t={co2_tstat:.3f}, p={co2_pval:.4f}")

if peak_kwh_pval < 0.05:
    print("PEAK kWh reduction is statistically significant (p < 0.05)")
else:
    print("PEAK kWh reduction is not statistically significant (p ≥ 0.05)")
    
if co2_pval < 0.05:
    print("FULL DAY CO2 reduction is statistically significant (p < 0.05)")
else:
    print("FULL DAY CO2 reduction is not statistically significant (p ≥ 0.05)")

# SUMMARY
# Calculate monthly reductions
monthly_peak_reduction = peak_effects['net_peak_kwh_reduction'].mean() * 30.44
monthly_co2_reduction = co2_effects['net_co2_reduction'].mean() * 30.44

# Calculate annual reductions  
annual_peak_reduction = monthly_peak_reduction * 12
annual_co2_reduction = monthly_co2_reduction * 12

# Calculate baselines differently to show variation
monthly_baseline_peak = control_baseline_peak_kwh.mean() * 30.44
annual_baseline_peak = control_baseline_peak_kwh.mean() * 365.25 
monthly_baseline_co2 = control_baseline_co2.mean() * 30.44  
annual_baseline_co2 = control_baseline_co2.mean() * 365.25

# Calculate percentage reductions
monthly_peak_pct = (monthly_peak_reduction / monthly_baseline_peak) * 100
monthly_co2_pct = (monthly_co2_reduction / monthly_baseline_co2) * 100

# Display results
print(f"\nMONTHLY REDUCTIONS per household:")
print(f"  Peak kWh: {monthly_peak_reduction:.1f} kWh/month ({monthly_peak_pct:.2f}%)")
print(f"  Full CO2: {monthly_co2_reduction/1000:.2f} kg CO2/month ({monthly_co2_pct:.2f}%)")

print(f"\nANNUAL REDUCTIONS per household:")
print(f"  Peak kWh: {annual_peak_reduction:.1f} kWh/year")
print(f"  Full CO2: {annual_co2_reduction/1000:.1f} kg CO2/year")


STATISTICAL SIGNIFICANCE
PEAK kWh reduction significance: t=-13.095, p=0.0000
FULL DAY CO2 reduction significance: t=-2.492, p=0.0148
PEAK kWh reduction is statistically significant (p < 0.05)
FULL DAY CO2 reduction is statistically significant (p < 0.05)

MONTHLY REDUCTIONS per household:
  Peak kWh: -6.4 kWh/month (-8.76%)
  Full CO2: -0.85 kg CO2/month (-1.93%)

ANNUAL REDUCTIONS per household:
  Peak kWh: -77.1 kWh/year
  Full CO2: -10.2 kg CO2/year


In [62]:
# best and worst performing months
print(f"\nBEST & WORST PERFORMING MONTHS")

best_peak_month = peak_effects.loc[peak_effects['net_peak_kwh_reduction'].idxmin()]
worst_peak_month = peak_effects.loc[peak_effects['net_peak_kwh_reduction'].idxmax()]

best_co2_month = co2_effects.loc[co2_effects['net_co2_reduction'].idxmin()] 
worst_co2_month = co2_effects.loc[co2_effects['net_co2_reduction'].idxmax()]

print(f"Peak kWh Reduction:")
print(f"  Best month: {best_peak_month['month_post']} ({best_peak_month['net_peak_kwh_reduction']:.3f} kWh/day)")
print(f"  Worst month: {worst_peak_month['month_post']} ({worst_peak_month['net_peak_kwh_reduction']:.3f} kWh/day)")

print(f"\nCO2 Reduction:")
print(f"  Best month: {best_co2_month['month_post']} ({best_co2_month['net_co2_reduction']:.1f} grams/day)")
print(f"  Worst month: {worst_co2_month['month_post']} ({worst_co2_month['net_co2_reduction']:.1f} grams/day)")

# Add variation insight
peak_variation = abs(best_peak_month['net_peak_kwh_reduction'] - worst_peak_month['net_peak_kwh_reduction'])
co2_variation = abs(best_co2_month['net_co2_reduction'] - worst_co2_month['net_co2_reduction'])


BEST & WORST PERFORMING MONTHS
Peak kWh Reduction:
  Best month: 2024-10 (-0.508 kWh/day)
  Worst month: 2024-09 (0.094 kWh/day)

CO2 Reduction:
  Best month: 2024-07 (-224.1 grams/day)
  Worst month: 2024-08 (153.5 grams/day)
